# Part 4

In this section, we extend the HMM to have second-order dependencies by modifying the Viterbi algorithm.

The logic is as follows:
Our Viterbi algorithm thus far uses a first-order assumption, only including the probability of the previous state in the calculation of the likelihood of the current state. We want to now include the *previous two states* in our calculation.

The algorithm can still be designed via dynamic programming. However, runtime is further multiplied by (number of states) due to the additional layer of multiplication, for a total runtime of $O(T \times |S|^3)$.

In [1]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm, trange
# from tqdm import tqdm_notebook as tqdm, tnrange as trange
from IPython.display import display

## Emissions
We train the emissions matrix by using maximum likelihood estimation:

$$ e(x|y) = \frac{Count(x->y)}{Count(y)} $$

In [4]:
# The behavior of the emissions matrix is the same - we use the same function as used in Part 3
def learn_emissions(train_filename):
    ''' Learns emissions parameters from data and returns them as a nested dictionary '''
    with open(train_filename, "r") as f:
        lines = f.readlines()

    observations = set()
    # Track emission counts
    emissions = {} # Where key is y, and value is a dictionary of emissions x from y with their frequency

    # Learn from data
    for line in tqdm(lines, desc='Emissions'):
        data_split = line.strip().rsplit(' ', 1)

        # Only process valid lines
        if len(data_split) == 2:
            obs, state = data_split
            observations.add(obs)

            # Track this emission
            if state in emissions:
                current_emissions = emissions[state]
            else:
                current_emissions = defaultdict(int)
                
            current_emissions[obs] += 1
            emissions[state] = current_emissions # Update
    
    emission_counts = {k: sum(emissions[k].values()) for k in emissions}
    return emissions, emission_counts, observations


def get_emission_parameters(emissions, emission_counts, observations, x, y, k=1):
    ''' Returns the MLE of the emission parameters based on the emissions dictionary '''
    if y not in emissions:  # edge case: no records of emission from this state
        return 0
    state_data = emissions[y]
    count_y = emission_counts[y] #sum(state_data.values()) # Denominator
    
    if x not in observations:  # edge case: no records of emission of this word
        count_y_x = k
    else:
        count_y_x = state_data[x]
    
    e = count_y_x / (count_y + k)
    return e

## Transitions

In [64]:
def learn_transitions(train_filename):
    """
    Returns a dictionary containing (key, value) where
        key: (u, v)
        value: Count(u, v)
    """
    with open(train_filename, 'r') as f:
        lines = f.readlines()
        
    transitions = defaultdict(int)
    prev_prev_state = 'PRESTART'
    prev_state = 'START'
    # avoid excessive indentations
    for line in tqdm(lines, desc='Transitions'):
        data_split = line.strip().rsplit(' ', 1)
        
        # line breaks -> new sequence
        if len(data_split) < 2:
            transitions[(prev_prev_state, prev_state, 'STOP')] += 1
            prev_prev_state = 'PRESTART'
            prev_state = 'START'
            continue

        obs, curr_state = data_split
        transitions[(prev_prev_state, prev_state, curr_state)] += 1
        prev_prev_state = prev_state
        prev_state = curr_state
        
    # count number of 'from' states
    transition_counts = defaultdict(int)
    for (t, u, v), counts in transitions.items():
        transition_counts[(t, u)] += counts

    # get all unique states
    t, u, v = zip(*transitions)
    states = set(t) | set(u) | set(v)
    return transitions, transition_counts, states

def get_transition_parameters(transitions, transition_counts, t, u, v):
    if transition_counts[(t, u)] == 0:  # edge case: no records of transitions starting from u
        return 0
    return transitions[(t, u, v)] / transition_counts[(t, u)]

We modify the Viterbi algorithm for Part 4, to include second-order dependencies.

### Viterbi

In [107]:
def viterbi(transitions, transition_counts, states, emissions, emission_counts, observations, obs_seq):
    a = lambda prev2, prev, curr: get_transition_parameters(transitions, transition_counts, prev2, prev, curr)
    b = lambda state, out: get_emission_parameters(emissions, emission_counts, observations, x=out, y=state)

    # create empty tables
    n = len(obs_seq) + 3  # PRESTART + START + |obs_seq| + STOP
    P = pd.DataFrame(index=states, columns=range(n)).fillna(0)  # probability table
    B = pd.DataFrame(index=states, columns=range(n))  # backpointer table
    
    # initialization
    P.loc['PRESTART', 0] = 1
    P.loc['START', 1] = 1
    
    # recursion
    for j in range(2, n-1):
        x = obs_seq[j-2]  # obs_seq starts from 0, j starts from 1
        for v in states:  # curr state
            for u in states:  # prev state
                for t in states: # prev_2 state
                    p = P.loc[u, j-1] * P.loc[t, j-2] * a(t, u, v) * b(v, x)
                    if p > P.loc[v, j]:
                        P.loc[v, j] = p  # update probability
                        B.loc[v, j] = t + ' ' + u  # update backpointer
                    
    # termination
    j = n - 1
    v = 'STOP'
    for u in states:
        for t in states:
            p = P.loc[u, j-1] * P.loc[t, j-2] * a(t, u, v)
            if p > P.loc[v, j]:
                P.loc[v, j] = p  # probability
                B.loc[v, j] = t + ' ' + u  # backpointer
    # backtrace
    # second order viterbi requires backpropagation keep track of the second order item, 
    # as the limited horizon assumption is now limited to *two* entries instead.
    state_seq = ['STOP']
    # Skip steps 2 at a time
    for i in range(n-1, 0, -2):
        curr_state = state_seq[-1]
        B_lookup = B.loc[curr_state, i]
        if isinstance(B_lookup, str):
            prev_state = B.loc[curr_state, i].split(' ')[1]
            prev_prev_state = B.loc[curr_state, i].split(' ')[0]
        else: # edge case: no possible transition to START
            for j in range(int(i/2)+1):
                state_seq.append(('O'))
                state_seq.append(('O'))
            break
        state_seq.append(prev_state)
        state_seq.append(prev_prev_state)
    state_seq = state_seq[::-1][2:-1]  # reverse and drop PRESTART, START, STOP
    return P, B, state_seq

In [108]:
def train(dataset):
    """
    Takes in dataset name, obtains transition and emission parameters.
    """
    train_filename = f'data/{dataset}/train'
    # train
    t = learn_transitions(train_filename)
    e = learn_emissions(train_filename)
    return t, e


def validate(dataset, t, e):
    """
    Takes in dataset name, and HMM parameters.
    Use HMM parameters to estimate state sequence.
    Write state sequence to file alongside input observation sequence.
    """
    val_filename = f'data/{dataset}/dev.in'
    out_filename = f'data/{dataset}/dev.p4.out'

    # validate with train parameters
    label_sequence = lambda sentence: viterbi(*t, *e, sentence)
    
    # read validation file for sequence
    with open(val_filename, 'r') as f:
        lines = f.readlines()

    sentence = []
    result = []
    for word in tqdm(lines, desc=dataset):
        if word == '\n':
            _, _, s = label_sequence(sentence)  # ignore tables
            result.extend(s)
            result.append('\n')
            sentence = []
        else:
            sentence.append(word.strip())
            
    # write sequence to out file
    with open(out_filename, 'w') as f:
        for i in range(len(lines)):
            word = lines[i].strip()
            if word:
                pred = result[i]
                f.write(word + ' ' + pred)
            f.write('\n')

In [109]:
datasets = ['EN', 'FR']
for dataset in datasets:
    t, e = train(dataset)
    validate(dataset, t, e)

FR: 100%|██████████| 3700/3700 [01:06<00:00, 57.28it/s]
